AE part

In [3]:
# read data from result_baseline/result_optimal.txt

with open('result_baseline/result_optimal_ae.txt', 'r') as f:
    file = f.read()
    datas = file.split('\n\n\n\n')


In [4]:
import re
from collections import defaultdict
from tabulate import tabulate

# Structuring the data
data_struct = defaultdict(lambda: defaultdict(dict))

for data in datas:
    # Extract the required information using regular expressions
    compare_name = re.search(r'Experiment settings: AE. (.+). Dataset', data).group(1)
    dataset_name = re.search(r'Dataset (.+)', data).group(1)

    # Extracting the mean and standard deviation for each metric
    AUC_ROC_mean, AUC_ROC_std = map(float, re.search(r'ROC-AUC: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    accuracy_mean, accuracy_std = map(float, re.search(r'Accuracy: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    sensitivity_mean, sensitivity_std = map(float, re.search(r'Sensitivity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    specificity_mean, specificity_std = map(float, re.search(r'Specificity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    # Significance ratio extraction is commented out as it seems not required currently.
    hz_para_list = list(map(int, re.search(r'hz_para_list: \[(.+)\]', data).group(1).split(', ')))

    # Organizing the data by compare_name and dataset
    data_struct[compare_name][dataset_name] = {
        'ROC-AUC': (AUC_ROC_mean, AUC_ROC_std),
        'Accuracy': (accuracy_mean, accuracy_std),
        'Sensitivity': (sensitivity_mean, sensitivity_std),
        'Specificity': (specificity_mean, specificity_std),
        # 'Significance Ratio': (significance_ratio_mean, significance_ratio_std) # Uncomment if needed
        'hz_para_list': hz_para_list,
        'compare_name': compare_name
    }

# change _ in compare_name to space
for compare_name, datasets in data_struct.items():
    for dataset_name, data in datasets.items():
        data['compare_name'] = data['compare_name'].replace('_', ' ')
    


# Printing the data in a latex format, only print ROC-AUC, Accuracy, Sensitivity, Specificity
for compare_name, datasets in data_struct.items():
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    compare_name = compare_name.replace('_', '-')
    print(f'\\caption{{Performance of AE in {compare_name} on different datasets}}')
    print(f'\\label{{tab:my-table}}')
    print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(datasets))]) + '}')
    print(f'\\hline')
    print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    metrics = ['ROC-AUC', 'Accuracy', 'Sensitivity', 'Specificity']
    highest_metrics = {}
    
    for dataset_name, data in datasets.items():    
        for metric in metrics:
            if metric not in highest_metrics:
                highest_metrics[metric] = (0, 0)
            if data[metric][0] > highest_metrics[metric][0]:
                highest_metrics[metric] = data[metric]
    for dataset_name, data in datasets.items():
        # find max and bold it
        for metric in metrics:
            if data[metric] == highest_metrics[metric]:
                data[metric] = (f'\\textbf{{{data[metric][0]:.2f}}}', f'\\textbf{{{data[metric][1]:.2f}}}')
            else:
                data[metric] = (f'{data[metric][0]:.2f}', f'{data[metric][1]:.2f}')
        print(f'{dataset_name} & ${data["ROC-AUC"][0]} \\pm {data["ROC-AUC"][1]}$ & ${data["Accuracy"][0]} \\pm {data["Accuracy"][1]}$ & ${data["Sensitivity"][0]} \\pm {data["Sensitivity"][1]}$ & ${data["Specificity"][0]} \\pm {data["Specificity"][1]}$ \\\\')
        
    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of AE in HC-vs-AD on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & $72.26 \pm 1.47$ & $77.04 \pm 3.38$ & $84.52 \pm 6.76$ & $56.58 \pm 6.89$ \\
fdg & $\textbf{85.74} \pm \textbf{1.00}$ & $77.18 \pm 4.19$ & $75.00 \pm 8.11$ & $\textbf{83.16} \pm \textbf{6.88}$ \\
vbm & $66.32 \pm 2.14$ & $64.86 \pm 8.66$ & $66.35 \pm 17.46$ & $60.79 \pm 15.90$ \\
Multimodal Mixture & $75.99 \pm 1.91$ & $72.78 \pm 2.64$ & $78.94 \pm 8.98$ & $65.25 \pm 7.49$ \\
Multimodal combine & $81.58 \pm 1.88$ & $77.29 \pm 1.79$ & $85.87 \pm 3.33$ & $66.82 \pm 4.55$ \\
Multimodal effect each & $79.60 \pm 1.94$ & $76.40 \pm 2.17$ & $87.21 \pm 5.54$ & $63.26 \pm 6.07$ \\
Multimodal (3+c) & $82.17 \pm 2.56$ & $\textbf{77.97} \pm \textbf{2.65}$ & $\textbf{87.69} \pm \textbf{5.54}$ & $66.37 \pm 7.20$ \\
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{tab

CVAE part

In [3]:
# read data from result_baseline/result_optimal.txt

with open('result_baseline/result_optimal_cvae.txt', 'r') as f:
    file = f.read()
    datas = file.split('\n\n\n\n')


In [8]:
import re
from collections import defaultdict
from tabulate import tabulate

# Structuring the data
data_struct = defaultdict(lambda: defaultdict(dict))

for data in datas:
    # Extract the required information using regular expressions
    compare_name = re.search(r'Experiment settings: CVAE. (.+). Dataset', data).group(1)
    dataset_name = re.search(r'Dataset (.+)', data).group(1)

    # Extracting the mean and standard deviation for each metric
    AUC_ROC_mean, AUC_ROC_std = map(float, re.search(r'ROC-AUC: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    accuracy_mean, accuracy_std = map(float, re.search(r'Accuracy: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    sensitivity_mean, sensitivity_std = map(float, re.search(r'Sensitivity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    specificity_mean, specificity_std = map(float, re.search(r'Specificity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    # Significance ratio extraction is commented out as it seems not required currently.
    hz_para_list = list(map(int, re.search(r'hz_para_list: \[(.+)\]', data).group(1).split(', ')))

    # Organizing the data by compare_name and dataset
    data_struct[compare_name][dataset_name] = {
        'ROC-AUC': (AUC_ROC_mean, AUC_ROC_std),
        'Accuracy': (accuracy_mean, accuracy_std),
        'Sensitivity': (sensitivity_mean, sensitivity_std),
        'Specificity': (specificity_mean, specificity_std),
        # 'Significance Ratio': (significance_ratio_mean, significance_ratio_std) # Uncomment if needed
        'hz_para_list': hz_para_list,
        'compare_name': compare_name
    }

# change _ in compare_name to space
for compare_name, datasets in data_struct.items():
    for dataset_name, data in datasets.items():
        data['compare_name'] = data['compare_name'].replace('_', ' ')
    


metrics = ['ROC-AUC', 'Accuracy', 'Sensitivity', 'Specificity']
# Printing the data in a latex format, only print ROC-AUC, Accuracy, Sensitivity, Specificity
for compare_name, datasets in data_struct.items():
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    compare_name = compare_name.replace('_', '-')
    print(f'\\caption{{Performance of CVAE in {compare_name} on different datasets}}')
    print(f'\\label{{tab:my-table}}')
    print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(metrics) + 1)]) + '}')
    print(f'\\hline')
    print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    metrics = ['ROC-AUC', 'Accuracy', 'Sensitivity', 'Specificity']
    highest_metrics = {}
    
    for dataset_name, data in datasets.items():    
        for metric in metrics:
            if metric not in highest_metrics:
                highest_metrics[metric] = (0, 0)
            if data[metric][0] > highest_metrics[metric][0]:
                highest_metrics[metric] = data[metric]
    for dataset_name, data in datasets.items():
        # find max and bold it
        for metric in metrics:
            if data[metric] == highest_metrics[metric]:
                data[metric] = (f'\\textbf{{{data[metric][0]:.2f}}}', f'\\textbf{{{data[metric][1]:.2f}}}')
            else:
                data[metric] = (f'{data[metric][0]:.2f}', f'{data[metric][1]:.2f}')
        print(f'{dataset_name} & ${data["ROC-AUC"][0]} \\pm {data["ROC-AUC"][1]}$ & ${data["Accuracy"][0]} \\pm {data["Accuracy"][1]}$ & ${data["Sensitivity"][0]} \\pm {data["Sensitivity"][1]}$ & ${data["Specificity"][0]} \\pm {data["Specificity"][1]}$ \\\\')
        
    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of CVAE in HC-vs-AD on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & $69.99 \pm 4.53$ & $67.88 \pm 4.05$ & $73.94 \pm 10.92$ & $60.56 \pm 8.09$ \\
fdg & $\textbf{73.74} \pm \textbf{3.87}$ & $69.08 \pm 3.46$ & $62.98 \pm 9.20$ & $\textbf{76.60} \pm \textbf{7.41}$ \\
vbm & $59.64 \pm 4.44$ & $61.31 \pm 3.41$ & $74.23 \pm 12.83$ & $45.73 \pm 14.74$ \\
3modalities & $71.09 \pm 4.21$ & $\textbf{69.71} \pm \textbf{2.83}$ & $\textbf{81.15} \pm \textbf{6.55}$ & $56.09 \pm 7.52$ \\
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of CVAE in HC-vs-MCI on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & $\textbf{55.30} \pm \textbf{2.74}$ & $49.51 \pm 9.71$ & $43.80 \pm 15.95$ & $\textbf{69.57} 

FAAE part

In [13]:
import re

In [1]:
file = open('result_focal/result_multimodal.txt', 'r')
with file as f:
    file = f.read()
    # case sensitive
    datas = file.split('\n\n\n\ne')
for i in range(len(datas)):
    if datas[i].startswith('xperiment'):
        datas[i] = 'e' + datas[i]

In [2]:
import re

alpha_focal = []
gamma_focal = []
compare_names = ['MCI_vs_AD', 'HC_vs_MCI', 'HC_vs_AD']
results = {}

for data in datas:
    # Extract alpha_focal and gamma_focal
    alpha_match = re.search(r'args\.alpha_focal: (.+)', data)
    gamma_match = re.search(r'args\.gamma_focal: (.+)', data)
    if alpha_match and gamma_match:
        alpha_focal.append(float(alpha_match.group(1)))
        gamma_focal.append(float(gamma_match.group(1)))

    # Extract compare_name results
    for compare_name in compare_names:
        compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset av45\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
        
        if compare_match:
            auc_roc_mean = float(compare_match.group(1))
            auc_roc_std = float(compare_match.group(2))
            accuracy_mean = float(compare_match.group(3))
            accuracy_std = float(compare_match.group(4))
            sensitivity_mean = float(compare_match.group(5))
            sensitivity_std = float(compare_match.group(6))
            specificity_mean = float(compare_match.group(7))
            specificity_std = float(compare_match.group(8))
            significance_ratio_mean = float(compare_match.group(9))
            significance_ratio_std = float(compare_match.group(10))
            hz_para_list = list(map(int, compare_match.group(11).split(', ')))

            if compare_name not in results:
                results[compare_name] = []

            results[compare_name].append({
                'auc_roc_mean': auc_roc_mean,
                'auc_roc_std': auc_roc_std,
                'accuracy_mean': accuracy_mean,
                'accuracy_std': accuracy_std,
                'sensitivity_mean': sensitivity_mean,
                'sensitivity_std': sensitivity_std,
                'specificity_mean': specificity_mean,
                'specificity_std': specificity_std,
                'significance_ratio_mean': significance_ratio_mean,
                'significance_ratio_std': significance_ratio_std,
                'hz_para_list': hz_para_list
            })


# help me extract information. for data in datas, the data format is like above, attract information. each data have one alpha_focal and gamma_focal. remeber them. and each data have 3  compare_name MCI_vs_AD HC_vs_MCI HC_vs_AD. under each compare_name,\ there are auc_roc, accuracy, sensitivity, specificity, significance_ratio, hz_para_list, GET THEM RESPECTIVELY.





alpha_focal = []
gamma_focal = []
compare_names = ['MCI_vs_AD', 'HC_vs_MCI', 'HC_vs_AD']
results = {}

for data in datas:
    # Extract alpha_focal and gamma_focal
    alpha_match = re.search(r'args\.alpha_focal: (.+)', data)
    gamma_match = re.search(r'args\.gamma_focal: (.+)', data)
    if alpha_match and gamma_match:
        alpha_focal.append(float(alpha_match.group(1)))
        gamma_focal.append(float(gamma_match.group(1)))

    # Extract compare_name results
    for compare_name in compare_names:
        compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset av45\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
        if compare_match:
            auc_roc_mean = float(compare_match.group(1))
            auc_roc_std = float(compare_match.group(2))
            accuracy_mean = float(compare_match.group(3))
            accuracy_std = float(compare_match.group(4))
            sensitivity_mean = float(compare_match.group(5))
            sensitivity_std = float(compare_match.group(6))
            specificity_mean = float(compare_match.group(7))
            specificity_std = float(compare_match.group(8))
            significance_ratio_mean = float(compare_match.group(9))
            significance_ratio_std = float(compare_match.group(10))
            hz_para_list = list(map(int, compare_match.group(11).split(', ')))

            if compare_name not in results:
                results[compare_name] = []

            results[compare_name].append({
                'dataset_name': 'av45', # 'av45' is hardcoded here, you can extract it from the data using regex
                'auc_roc_mean': auc_roc_mean,
                'auc_roc_std': auc_roc_std,
                'accuracy_mean': accuracy_mean,
                'accuracy_std': accuracy_std,
                'sensitivity_mean': sensitivity_mean,
                'sensitivity_std': sensitivity_std,
                'specificity_mean': specificity_mean,
                'specificity_std': specificity_std,
                'significance_ratio_mean': significance_ratio_mean,
                'significance_ratio_std': significance_ratio_std,
                'hz_para_list': hz_para_list,
                'compare_name': compare_name,
                'alpha_focal': float(alpha_match.group(1)),
                'gamma_focal': float(gamma_match.group(1))
            })
        else:
            # compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset (.+) \nauc_roc: mean\n(.+)\naccuracy: mean\n(.+)\nsensitivity: mean\n(.+)\nspecificity: mean\n(.+)\nsignificance_ratio: mean\n(.+)\nhz_para_list: \[(.+)\]', data)
            compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset (.+)\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
            if compare_match:
                dataset_name = compare_match.group(1)
                auc_roc_mean = float(compare_match.group(2))
                auc_roc_std = float(compare_match.group(3))
                accuracy_mean = float(compare_match.group(4))
                accuracy_std = float(compare_match.group(5))
                sensitivity_mean = float(compare_match.group(6))
                sensitivity_std = float(compare_match.group(7))
                specificity_mean = float(compare_match.group(8))
                specificity_std = float(compare_match.group(9))
                significance_ratio_mean = float(compare_match.group(10))
                significance_ratio_std = float(compare_match.group(11))
                hz_para_list = list(map(int, compare_match.group(12).split(', ')))

                if compare_name not in results:
                    results[compare_name] = []

                results[compare_name].append({
                    'dataset_name': dataset_name,
                    'auc_roc_mean': auc_roc_mean,
                    'auc_roc_std': auc_roc_std,
                    'accuracy_mean': accuracy_mean,
                    'accuracy_std': accuracy_std,
                    'sensitivity_mean': sensitivity_mean,
                    'sensitivity_std': sensitivity_std,
                    'specificity_mean': specificity_mean,
                    'specificity_std': specificity_std,
                    'significance_ratio_mean': significance_ratio_mean,
                    'significance_ratio_std': significance_ratio_std,
                    'hz_para_list': hz_para_list,
                    'compare_name': compare_name,
                    'alpha_focal': float(alpha_match.group(1)),
                    'gamma_focal': float(gamma_match.group(1))
                    
                })

            else:
                print(f'No match found for compare_name: {compare_name}')




No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD
No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD
No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD
No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD
No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD
No match found for compare_name: MCI_vs_AD
No match found for compare_name: HC_vs_MCI
No match found for compare_name: HC_vs_AD


In [24]:
new_results = {}
for com in results:
    new_results[com] = [item for item in results[com] if item['hz_para_list'] == [20, 20]]
results = new_results



In [17]:
# if gamma is 30 or alpha is 1, remove the data
new_results = {}
for com in results:
    new_results[com] = [item for item in results[com] if (item['gamma_focal'] != 30 and item['alpha_focal'] != 1 and item['alpha_focal'] != 0.7)]
results = new_results


In [18]:
# multiply all number values by 100
for compare_name, compare_results in results.items():
    for result in compare_results:
        result['auc_roc_mean'] *= 100
        result['auc_roc_std'] *= 100
        result['accuracy_mean'] *= 100
        result['accuracy_std'] *= 100
        result['sensitivity_mean'] *= 100
        result['sensitivity_std'] *= 100
        result['specificity_mean'] *= 100
        result['specificity_std'] *= 100
        result['significance_ratio_mean'] *= 100
        result['significance_ratio_std'] *= 100
        result['alpha_focal'] = float(result['alpha_focal'])
        result['gamma_focal'] = float(result['gamma_focal'])

In [20]:
# for each compare_name, print a latex table

datasets = ['av45', 'fdg', 'vbm', 'Multimodal']

for compare_name, compare_results in results.items():    
    for dataset_name in datasets:
        print(f'\\begin{{table}}[H]')
        print(f'\\centering')
        compare_name = compare_name.replace('_', '-')
        print(f'\\caption{{Performance of {compare_name} on {dataset_name} datasets}}')
        print(f'\\label{{tab:my-table}}')
        # print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(compare_results) + 1)]) + '}')
        # print('') \begin{tabular}{c|c|c|c|c|c|c}
        print(f'\\begin{{tabular}}{{c|c|c|c|c}}')
        print(f'\\hline')
        # add alpha and gamma
        # print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
        print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
        print(f'\\hline')
        for result in compare_results:
            if result['dataset_name'] != dataset_name:
                continue
            print(f'{result["dataset_name"]} & ${result["auc_roc_mean"]:.2f} \\pm {result["auc_roc_std"]:.2f}$ & ${result["accuracy_mean"]:.2f} \\pm {result["accuracy_std"]:.2f}$ & ${result["sensitivity_mean"]:.2f} \\pm {result["sensitivity_std"]:.2f}$ & ${result["specificity_mean"]:.2f} \\pm {result["specificity_std"]:.2f}$ \\\\')
        print(f'\\hline')
        print(f'\\end{{tabular}}')
        print(f'\\end{{table}}')
        print(f'\\vspace{{1cm}}')
        print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on av45 datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on fdg datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on vbm datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on Multimodal datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
\hline
\end{tabular}

In [322]:
# for each alpha and gamma and dataset pair, get a averge on compare_name

data1, data2, data3 = results['MCI_vs_AD'], results['HC_vs_MCI'], results['HC_vs_AD']



In [323]:
def ifdatassharesame(property):
    for i in range(len(data1)):
        # if data1[i][property] != data2[i][property] or data1[i][property] != data3[i][property]:
        if data1[i][property] == data2[i][property] and data1[i][property] == data3[i][property]:            
            continue
        else:
            print(data1[i][property], data2[i][property], data3[i][property])
    return True

In [324]:
if ifdatassharesame('dataset_name') and ifdatassharesame('alpha_focal') and ifdatassharesame('gamma_focal') and ifdatassharesame('hz_para_list'):
    print('all dataset names are the same')

all dataset names are the same


In [325]:
# create a new list, which get the average of the three compare_names

average_results = []

for i in range(len(data1)):
    average_results.append({
        'dataset_name': data1[i]['dataset_name'],
        'alpha_focal': data1[i]['alpha_focal'],
        'gamma_focal': data1[i]['gamma_focal'],
        'hz_para_list': data1[i]['hz_para_list'],
        'auc_roc_mean': (data1[i]['auc_roc_mean'] + data2[i]['auc_roc_mean'] + data3[i]['auc_roc_mean']) / 3,
        'auc_roc_std': (data1[i]['auc_roc_std'] + data2[i]['auc_roc_std'] + data3[i]['auc_roc_std']) / 3,
        'accuracy_mean': (data1[i]['accuracy_mean'] + data2[i]['accuracy_mean'] + data3[i]['accuracy_mean']) / 3,
        'accuracy_std': (data1[i]['accuracy_std'] + data2[i]['accuracy_std'] + data3[i]['accuracy_std']) / 3,
        'sensitivity_mean': (data1[i]['sensitivity_mean'] + data2[i]['sensitivity_mean'] + data3[i]['sensitivity_mean']) / 3,
        'sensitivity_std': (data1[i]['sensitivity_std'] + data2[i]['sensitivity_std'] + data3[i]['sensitivity_std']) / 3,
        'specificity_mean': (data1[i]['specificity_mean'] + data2[i]['specificity_mean'] + data3[i]['specificity_mean']) / 3,
        'specificity_std': (data1[i]['specificity_std'] + data2[i]['specificity_std'] + data3[i]['specificity_std']) / 3,
        'significance_ratio_mean': (data1[i]['significance_ratio_mean'] + data2[i]['significance_ratio_mean'] + data3[i]['significance_ratio_mean']) / 3,
        'significance_ratio_std': (data1[i]['significance_ratio_std'] + data2[i]['significance_ratio_std'] + data3[i]['significance_ratio_std']) / 3
    })


In [326]:
datasets = ['av45', 'fdg', 'vbm', 'snp']

for dataset_name in datasets:
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    print(f'\\caption{{Performance of 3 average comparison on {dataset_name} datasets}}')
    print(f'\\label{{tab:my-table}}')
    # print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(compare_results) + 1)]) + '}')
    # print('') \begin{tabular}{c|c|c|c|c|c|c}
    print(f'\\begin{{tabular}}{{c|c|c|c|c|c|c}}')
    print(f'\\hline')
    # add alpha and gamma
    # print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    for result in average_results:
        if result['dataset_name'] != dataset_name:
            continue
        print(f'{result["alpha_focal"]} & {result["gamma_focal"]} & {result["dataset_name"]} & ${result["auc_roc_mean"]:.2f} \\pm {result["auc_roc_std"]:.2f}$ & ${result["accuracy_mean"]:.2f} \\pm {result["accuracy_std"]:.2f}$ & ${result["sensitivity_mean"]:.2f} \\pm {result["sensitivity_std"]:.2f}$ & ${result["specificity_mean"]:.2f} \\pm {result["specificity_std"]:.2f}$ \\\\')
    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of 3 average comparison on av45 datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c|c|c}
\hline
alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
0.1 & 1.0 & av45 & $57.04 \pm 2.26$ & $52.58 \pm 7.76$ & $64.42 \pm 13.02$ & $52.56 \pm 11.80$ \\
0.1 & 5.0 & av45 & $56.59 \pm 2.47$ & $54.04 \pm 5.63$ & $64.35 \pm 9.96$ & $52.87 \pm 8.63$ \\
0.1 & 10.0 & av45 & $56.75 \pm 1.93$ & $53.99 \pm 4.67$ & $62.62 \pm 8.43$ & $54.43 \pm 8.55$ \\
0.1 & 20.0 & av45 & $56.74 \pm 1.69$ & $54.28 \pm 4.90$ & $65.49 \pm 7.67$ & $52.40 \pm 7.32$ \\
0.25 & 1.0 & av45 & $56.87 \pm 1.54$ & $54.41 \pm 6.16$ & $63.11 \pm 9.28$ & $54.31 \pm 8.23$ \\
0.25 & 5.0 & av45 & $57.28 \pm 2.41$ & $53.45 \pm 7.05$ & $61.90 \pm 10.84$ & $56.60 \pm 9.68$ \\
0.25 & 10.0 & av45 & $57.41 \pm 1.92$ & $56.14 \pm 5.52$ & $65.85 \pm 9.41$ & $53.73 \pm 8.96$ \\
0.25 & 20.0 & av45 & $56.82 \pm 1.98$ & $55.54 \pm 6.14$ & $65.22 \pm 9.78$ & $52.41 \

Multimodal part

In [18]:
# read data from result_baseline/result_optimal.txt

with open('result_focal/result_for_table.txt', 'r') as f:
    file = f.read()
    datas = file.split('\n\n\n\n')


In [20]:
import re
from collections import defaultdict
from tabulate import tabulate

# Structuring the data
data_struct = defaultdict(lambda: defaultdict(dict))

for data in datas:
    # Extract the required information using regular expressions
    compare_name = re.search(r'Experiment settings: FAAE. (.+). Dataset', data).group(1)
    dataset_name = re.search(r'Dataset (.+)', data).group(1)

    # Extracting the mean and standard deviation for each metric
    AUC_ROC_mean, AUC_ROC_std = map(float, re.search(r'ROC-AUC: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    accuracy_mean, accuracy_std = map(float, re.search(r'Accuracy: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    sensitivity_mean, sensitivity_std = map(float, re.search(r'Sensitivity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    specificity_mean, specificity_std = map(float, re.search(r'Specificity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    # Significance ratio extraction is commented out as it seems not required currently.
    hz_para_list = list(map(int, re.search(r'hz_para_list: \[(.+)\]', data).group(1).split(', ')))

    # Organizing the data by compare_name and dataset
    data_struct[compare_name][dataset_name] = {
        'ROC-AUC': (AUC_ROC_mean, AUC_ROC_std),
        'Accuracy': (accuracy_mean, accuracy_std),
        'Sensitivity': (sensitivity_mean, sensitivity_std),
        'Specificity': (specificity_mean, specificity_std),
        # 'Significance Ratio': (significance_ratio_mean, significance_ratio_std) # Uncomment if needed
        'hz_para_list': hz_para_list,
        'compare_name': compare_name
    }

# change _ in compare_name to space
for compare_name, datasets in data_struct.items():
    for dataset_name, data in datasets.items():
        data['compare_name'] = data['compare_name'].replace('_', ' ')
    

metrics = ['ROC-AUC', 'Accuracy', 'Sensitivity', 'Specificity']
# Printing the data in a latex format, only print ROC-AUC, Accuracy, Sensitivity, Specificity
for compare_name, datasets in data_struct.items():
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    compare_name = compare_name.replace('_', '-')
    print(f'\\caption{{Performance of FAAE in {compare_name} on different datasets}}')
    print(f'\\label{{tab:my-table}}')
    print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(metrics) + 1 )]) + '}')
    print(f'\\hline')
    print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    
    
    highest_metrics = {}
    
    for dataset_name, data in datasets.items():    
        for metric in metrics:
            if metric not in highest_metrics:
                highest_metrics[metric] = (0, 0)
            if data[metric][0] > highest_metrics[metric][0]:
                highest_metrics[metric] = data[metric]
        
    for dataset_name, data in datasets.items():    
        # find max and bold it
        for metric in metrics:
            if data[metric] == highest_metrics[metric]:
                data[metric] = (f'\\textbf{{{data[metric][0]:.2f}}}', f'\\textbf{{{data[metric][1]:.2f}}}')
            else:
                data[metric] = (f'{data[metric][0]:.2f}', f'{data[metric][1]:.2f}')
        print(f'{dataset_name} & ${data["ROC-AUC"][0]} \\pm {data["ROC-AUC"][1]}$ & ${data["Accuracy"][0]} \\pm {data["Accuracy"][1]}$ & ${data["Sensitivity"][0]} \\pm {data["Sensitivity"][1]}$ & ${data["Specificity"][0]} \\pm {data["Specificity"][1]}$ \\\\')
        
        

    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of FAAE in HC-vs-AD on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & $61.23 \pm 8.34$ & $59.69 \pm 5.88$ & $50.48 \pm 21.25$ & $\textbf{71.19} \pm \textbf{15.98}$ \\
fdg & $64.96 \pm 5.14$ & $64.19 \pm 3.28$ & $69.23 \pm 14.74$ & $58.31 \pm 17.43$ \\
vbm & $55.65 \pm 2.91$ & $59.32 \pm 1.84$ & $71.63 \pm 15.84$ & $44.44 \pm 16.72$ \\
Multimodal MoE & $61.80 \pm 6.19$ & $62.27 \pm 3.53$ & $68.75 \pm 8.78$ & $54.32 \pm 14.64$ \\
Multimodal combining & $\textbf{71.53} \pm \textbf{4.35}$ & $\textbf{70.17} \pm \textbf{3.98}$ & $\textbf{80.48} \pm \textbf{6.66}$ & $57.82 \pm 8.07$ \\
Multimodal PoE & $62.81 \pm 5.15$ & $63.32 \pm 3.54$ & $71.35 \pm 13.48$ & $54.01 \pm 12.72$ \\
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of FAAE in HC-vs-MCI on different datasets}
\label{tab:my-tabl